In [6]:
import pandas as pd 
df = pd.read_csv("LE.csv")

In [20]:
from catboost import CatBoostRegressor, Pool
!pip install catboost
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
    --------------------------------------- 3.7/150.0 MB 27.3 MB/s eta 0:00:06
   --- ------------------------------------ 13.6/150.0 MB 40.8 MB/s eta 0:00:04
   ------ --------------------------------- 26.0/150.0 MB 48.4 MB/s eta 0:00:03
   ---------- ----------------------------- 38.0/150.0 MB 51.4 MB/s eta 0:00:03
   -------------- ------------------------- 54.8/150.0 MB 57.2 MB/s eta 0:00:02
   ------------------- -------------------- 72.1/150.0 MB 61.3 MB/s eta 0:00:02
   ----------------------- ---------------- 89.7/150.0 MB 65.0 MB/s eta 0:00:01
   --------------------------- ----------- 107.2/150.0 MB 67.1 MB/s eta 0:00:01
   -------------------------------- ------ 125.0/150.0 MB 69.4 MB/s eta 0:00:01
   ------------------------------------ -- 140.5/150.0 MB 70.1 MB/s eta 0:00:01
   --------------------------------------  149.9/150.0 MB 71.0 MB/s eta 0:00:01
   --------------------------------------  149.9/1

In [12]:
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [14]:
pool_train = Pool(X_train, y_train, cat_features = ['Country','Year','Status'])

In [15]:
pool_test = Pool(X_test, cat_features = ['Country','Year','Status'])


#CatBoost

In [17]:
import numpy as np

In [18]:
import time

start = time.time()

cbr = CatBoostRegressor(iterations=100,max_depth=2)

cbr.fit(pool_train)
y_pred = cbr.predict(X_test)
from sklearn.metrics import r2_score as RSquared


cb_rsquared = np.sqrt(RSquared(y_test, y_pred))
print("R Squared for CatBoost: ", np.mean(cb_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)

Learning rate set to 0.304491
0:	learn: 7.9025775	total: 15.9ms	remaining: 1.57s
1:	learn: 6.6171156	total: 39.8ms	remaining: 1.95s
2:	learn: 5.7048219	total: 52.1ms	remaining: 1.68s
3:	learn: 5.0759920	total: 73.9ms	remaining: 1.77s
4:	learn: 4.6535935	total: 85.5ms	remaining: 1.62s
5:	learn: 4.3251864	total: 106ms	remaining: 1.66s
6:	learn: 4.0517530	total: 127ms	remaining: 1.68s
7:	learn: 3.8478164	total: 149ms	remaining: 1.72s
8:	learn: 3.6603305	total: 168ms	remaining: 1.7s
9:	learn: 3.5540215	total: 184ms	remaining: 1.66s
10:	learn: 3.4172624	total: 203ms	remaining: 1.64s
11:	learn: 3.3573197	total: 226ms	remaining: 1.66s
12:	learn: 3.2889161	total: 248ms	remaining: 1.66s
13:	learn: 3.2534722	total: 264ms	remaining: 1.62s
14:	learn: 3.2195159	total: 281ms	remaining: 1.59s
15:	learn: 3.1754573	total: 298ms	remaining: 1.56s
16:	learn: 3.1537368	total: 314ms	remaining: 1.53s
17:	learn: 3.1129730	total: 330ms	remaining: 1.5s
18:	learn: 3.0919768	total: 346ms	remaining: 1.47s
19:	lear

In [21]:
import xgboost as xgb
from sklearn import preprocessing
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']

lbl = preprocessing.LabelEncoder()
#Country','Year','Status
X['Country'] = lbl.fit_transform(X['Country'].astype(str))
X['Year'] = lbl.fit_transform(X['Year'].astype(str))
X['Status'] = lbl.fit_transform(X['Status'].astype(str))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

start = time.time()
#X_train["Species"].astype("category")
xgbr = xgb.XGBRegressor()

xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)

xgb_rmse = np.sqrt(RSquared(y_test, y_pred))
print("R Squared for XGBoost: ", np.mean(xgb_rmse))

end = time.time()
diff = end - start
print('Execution time:', diff)

R Squared for XGBoost:  0.9796591712933758
Execution time: 0.4153261184692383


In [24]:
import lightgbm
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']
obj_feat = list(X.loc[:, X.dtypes == 'object'].columns.values)

for feature in obj_feat:
    X[feature] = pd.Series(X[feature], dtype="category")

start = time.time()

lgbmr = lightgbm.LGBMRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

lgbmr.fit(X_train, y_train)
y_pred = lgbmr.predict(X_test)

lgbm_rsquared = np.sqrt(RSquared(y_test, y_pred))
print()
print("R squared for LightGBM: ", np.mean(lgbm_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000527 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3522
[LightGBM] [Info] Number of data points in the train set: 2342, number of used features: 21
[LightGBM] [Info] Start training from score 69.262724

R squared for LightGBM:  0.9829598720069529
Execution time: 0.42711687088012695


In [23]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 9.5 MB/s eta 0:00:00
